In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import APIFetcher as API

In [5]:
n=100
snapfifty = API.fetch_historical_5m(n)

c:\Users\Mstav\RunescapeGET\RunescapeGET\ApiFetcher.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_t], ignore_index=True)


In [ ]:
#grab ID's with rows with NaN values == TRUE
NAindex=snapfifty[snapfifty.isna().any(axis=1)]['item_id'].unique()
snapfiftyprune = snapfifty[~snapfifty["item_id"].isin(NAindex)]
#removing low volume items
snapfiftyprunegroup= snapfiftyprune.groupby('item_id').nunique()
#n-1 to ensure proper ranging
filtered_indexes = snapfiftyprunegroup[snapfiftyprunegroup['timestamp'] != n-1].index
snapfiftyprune=snapfiftyprune[~snapfiftyprune['item_id'].isin(filtered_indexes)]
#rechecking groupings for n timestamps
snapfiftyprunegroup= snapfiftyprune.groupby('item_id').nunique()
#Weighted average of High/Low Price by High/Low Volume
snapfiftyprune['totalvol']=snapfiftyprune['highPriceVolume']+snapfiftyprune['lowPriceVolume']
snapfiftyprune['wprice']=(snapfiftyprune['highPriceVolume']/snapfiftyprune['totalvol'])*(snapfiftyprune['avgHighPrice']-snapfiftyprune['avgLowPrice'])+snapfiftyprune['avgLowPrice']
#transforming panel data to price and volume matrices
price_matrix_snapfifty=snapfiftyprune.pivot(index="timestamp", columns="item_id", values="wprice")
vol_matrix_snapfifty=snapfiftyprune.pivot(index='timestamp', columns='item_id', values='totalvol')
corr_price_snapfifty=price_matrix_snapfifty.corr()
corr_vol_snapfifty=vol_matrix_snapfifty.corr()
print(corr_price_snapfifty[corr_price_snapfifty > 0.7])


item_id  12934      1605  199         2  21326  21820  21902  231  2353  3024  \
item_id                                                                         
12934      1.0       NaN  NaN       NaN    NaN    NaN    NaN  NaN   NaN   NaN   
1605       NaN  1.000000  NaN  0.723344    NaN    NaN    NaN  NaN   NaN   NaN   
199        NaN       NaN  1.0       NaN    NaN    NaN    NaN  NaN   NaN   NaN   
2          NaN  0.723344  NaN  1.000000    NaN    NaN    NaN  NaN   NaN   NaN   
21326      NaN       NaN  NaN       NaN    1.0    NaN    NaN  NaN   NaN   NaN   
21820      NaN       NaN  NaN       NaN    NaN    1.0    NaN  NaN   NaN   NaN   
21902      NaN       NaN  NaN       NaN    NaN    NaN    1.0  NaN   NaN   NaN   
231        NaN       NaN  NaN       NaN    NaN    NaN    NaN  1.0   NaN   NaN   
2353       NaN       NaN  NaN       NaN    NaN    NaN    NaN  NaN   1.0   NaN   
3024       NaN       NaN  NaN       NaN    NaN    NaN    NaN  NaN   NaN   1.0   
440        NaN  0.792155  Na

In [31]:
#volatility
volatilityitems = price_matrix_snapfifty.rolling(window=4).std()
volatilitymarket= volatilityitems.sum(axis=1)
#Dividing by shape as count of row/column length
volatilitymarket=volatilitymarket/price_matrix_snapfifty.shape[1]
volatilitymarket
print(snapfiftyprune['totalvol'].shape)

(2079,)


In [ ]:
#440 1605 item id

features= vol_matrix_snapfifty['1605'].iloc[0:98]
features=pd.concat([features, price_matrix_snapfifty["1605"]], axis=1).iloc[0:98]
features=pd.concat([features, vol_matrix_snapfifty["440"]], axis=1).iloc[0:98]
features=pd.concat([features, price_matrix_snapfifty["440"]], axis=1).iloc[0:98]
features.columns=['ID 1605 Volume','ID 1605 Price','ID 440 Volume','ID 440 Price']
target=price_matrix_snapfifty['440'].iloc[1:100]

,ID 1605 Volume,ID 1605 Price,ID 440 Volume,ID 440 Price
timestamp,,,,
1746891900,1783,398.9871,18981,176.21158
1746892200,1070,395.238318,15177,176.002108
1746892500,978,392.501022,23141,178.299468
1746892800,793,392.645649,17729,177.594901
1746893100,1859,395.374933,73980,175.0
...,...,...,...,...
1746919800,16876,411.067552,19545,181.41888
1746920100,10539,413.437328,9943,182.973549
1746920400,11316,410.929215,34485,181.675337


timestamp
1746892200    176.002108
1746892500    178.299468
1746892800    177.594901
1746893100         175.0
1746893400    173.915986
                 ...    
1746920100    182.973549
1746920400    181.675337
1746920700    182.240459
1746921000    181.807451
1746921300    182.184906
Name: 440, Length: 98, dtype: object

In [ ]:
#440 1605 item id


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)
print(y_pred)

y_pred = rf_model.predict(X_test)  # Predicted values
df_pred = pd.DataFrame({"timestamp": df_actual["timestamp"], "predicted_price": y_pred})

[179.37625576 173.82290544 181.75382353 172.00138672 178.39924782
 178.69804578 178.96279305 173.82773248 172.1598382  177.67721234
 172.23621896 177.3190424  181.18358435 172.36013213 173.86261602
 173.32505458 172.17042635 172.08220509 179.31103787 179.42657678]
